## Setup

In [2]:
source("/research/lab_reiberger/2022_PSVD_metabolomics/2022_PSVD_metabolomics/R/00_setup.R")

In [3]:
p_load("dplyr", "ggplot2", "caret", "mlbench", install = FALSE)

In [4]:
metabol <- read.csv("outputs/01_metabolomics_merged.csv")
metabol <- metabol[2:ncol(metabol)]
metadata <- read.csv("outputs/01_metadata.csv")
metadata <- metadata[2:ncol(metadata)]

## Feature selection

In [42]:
metadata_psvd_hc <- metadata %>% filter(group_123 %in% c(2, 3))

In [43]:
metadata_psvd_hc <- metadata_psvd_hc %>% dplyr::select(-sample_description)

In [41]:
metabol <- metabol %>% select_if(~ !any(is.na(.)))

In [44]:
metadata_psvd_hc <- merge(metadata_psvd_hc, metabol, by = "sample_id")

In [45]:
metadata_psvd_hc$group_123 <- as.factor(metadata_psvd_hc$group_123)
x_train <- metadata_psvd_hc[,-2]
y_train <- metadata_psvd_hc[, 2]
     

In [46]:
control_rfe_disease_staging = rfeControl(functions = nbFuncs, # random forest
                      method = "repeatedcv", # repeated cv
                      repeats = 5, # number of repeats
                      number = 10) # number of folds

set.seed(42)
# Performing RFE
defaultW <- getOption("warn") 
options(warn = -1)
result_rfe_disease_staging = rfe(x = x_train %>% dplyr::select(-sample_id), 
                 y = y_train, 
                 sizes = c(1:10),
                 rfeControl = control_rfe_disease_staging)

# result_rfe_disease_staging = train(x = x_train_disease_staging %>% dplyr::select(-Sample_id), 
#                  method = "rf",
#                  y = y_train_disease_staging, 
#                  sizes = c(1:10),
#                  trControl = control_rfe_disease_staging)

# summarising the results
result_rfe_disease_staging
options(warn = defaultW)


Recursive feature selection

Outer resampling method: Cross-Validated (10 fold, repeated 5 times) 

Resampling performance over subset size:

 Variables Accuracy Kappa AccuracySD KappaSD Selected
         1    0.905  0.81    0.13258  0.2652         
         2    0.920  0.84    0.12817  0.2563         
         3    0.925  0.85    0.11573  0.2315         
         4    0.930  0.86    0.11339  0.2268         
         5    0.930  0.86    0.13401  0.2680         
         6    0.950  0.90    0.11294  0.2259         
         7    0.950  0.90    0.12372  0.2474         
         8    0.945  0.89    0.12667  0.2533         
         9    0.945  0.89    0.12667  0.2533         
        10    0.955  0.91    0.10938  0.2188        *
       264    0.955  0.91    0.09702  0.1940         

The top 5 variables (out of 10):
   cer_d34_1, hydroxy_glutamic_acid, glutamic_acid, pyruvic_acid, citramalic_acid


In [47]:
result_rfe_disease_staging$optVariables

[1] "cer_d34_1"             "hydroxy_glutamic_acid" "glutamic_acid"        
 [4] "pyruvic_acid"          "citramalic_acid"       "ps_41_6"              
 [7] "deoxyuridine"          "pc_o_32_0"             "aconitic_acid"        
[10] "ce_20_4"

In [49]:
result_rfe_disease_staging$optVariables %>% as.vector()

[1] "cer_d34_1"             "hydroxy_glutamic_acid" "glutamic_acid"        
 [4] "pyruvic_acid"          "citramalic_acid"       "ps_41_6"              
 [7] "deoxyuridine"          "pc_o_32_0"             "aconitic_acid"        
[10] "ce_20_4"

In [50]:

roc_control = trainControl(method = 'LOOCV',                
                        number = 1,
                        seed = as.list(rep(1,70)),
                         savePredictions = 'final',        
                        ) # number of folds


roc_pred_stage <- train(group_123 ~ cer_d34_1 + hydroxy_glutamic_acid + glutamic_acid + pyruvic_acid + citramalic_acid + ps_41_6 + deoxyuridine + pc_o_32_0 + aconitic_acid + ce_20_4, 
                 data = metadata_psvd_hc,
                 method = "rf",
                 trControl = roc_control,
                 tuneGrid=data.frame(mtry=1))
roc_pred_stage
     

Random Forest 

40 samples
10 predictors
 2 classes: '2', '3' 

No pre-processing
Resampling: Leave-One-Out Cross-Validation 
Summary of sample sizes: 39, 39, 39, 39, 39, 39, ... 
Resampling results:

  Accuracy  Kappa
  0.975     0.95 

Tuning parameter 'mtry' was held constant at a value of 1

### PSVD vs cir

In [52]:
metadata_psvd_cir <- metadata %>% filter(group_123 %in% c(1, 2))
metadata_psvd_cir <- metadata_psvd_cir %>% dplyr::select(-sample_description)
metabol <- metabol %>% select_if(~ !any(is.na(.)))
metadata_psvd_cir <- merge(metadata_psvd_cir, metabol, by = "sample_id")
metadata_psvd_cir$group_123 <- as.factor(metadata_psvd_cir$group_123)
x_train <- metadata_psvd_cir[,-2]
y_train <- metadata_psvd_cir[, 2]
     
control_rfe_disease_staging = rfeControl(functions = nbFuncs, # random forest
                      method = "repeatedcv", # repeated cv
                      repeats = 5, # number of repeats
                      number = 10) # number of folds

set.seed(42)
# Performing RFE
defaultW <- getOption("warn") 
options(warn = -1)
result_rfe_disease_staging = rfe(x = x_train %>% dplyr::select(-sample_id), 
                 y = y_train, 
                 sizes = c(1:10),
                 rfeControl = control_rfe_disease_staging)

# result_rfe_disease_staging = train(x = x_train_disease_staging %>% dplyr::select(-Sample_id), 
#                  method = "rf",
#                  y = y_train_disease_staging, 
#                  sizes = c(1:10),
#                  trControl = control_rfe_disease_staging)

# summarising the results
result_rfe_disease_staging
options(warn = defaultW)
result_rfe_disease_staging$optVariables
result_rfe_disease_staging$optVariables %>% as.vector()

     


Recursive feature selection

Outer resampling method: Cross-Validated (10 fold, repeated 5 times) 

Resampling performance over subset size:

 Variables Accuracy Kappa AccuracySD KappaSD Selected
         1   0.5533 0.116     0.2713  0.5366         
         2   0.5783 0.150     0.2779  0.5611         
         3   0.6433 0.272     0.2936  0.6051         
         4   0.6167 0.214     0.2792  0.5792         
         5   0.6083 0.202     0.2516  0.5220         
         6   0.5767 0.134     0.2703  0.5594         
         7   0.5900 0.162     0.2421  0.5066         
         8   0.5550 0.092     0.2601  0.5386         
         9   0.5567 0.094     0.2659  0.5494         
        10   0.5867 0.154     0.2624  0.5448         
       264   0.7433 0.474     0.2239  0.4628        *

The top 5 variables (out of 264):
   adipic_acid, tyrosine, pc_32_1, tag_50_2, pe_40_4


[1] "adipic_acid"                                   
  [2] "tyrosine"                                      
  [3] "pc_32_1"                                       
  [4] "tag_50_2"                                      
  [5] "pe_40_4"                                       
  [6] "lac_cer_d42_2"                                 
  [7] "tag_48_2"                                      
  [8] "sm_d34_0"                                      
  [9] "aspartic_acid"                                 
 [10] "tag_48_0"                                      
 [11] "pc_32_0"                                       
 [12] "sm_d36_2"                                      
 [13] "tag_50_3"                                      
 [14] "pe_38_2"                                       
 [15] "taurocholic_acid"                              
 [16] "kynurenic_acid"                                
 [17] "deoxyuridine"                                  
 [18] "lpc_20_4"                                      
 [19] "pc_35_2"                                       
 [20] "xanthosine"                                    
 [21] "lpc_18_2"                                      
 [22] "tag_48_1"                                      
 [23] "phenylpyruvic_acid"                            
 [24] "sm_d34_2"                                      
 [25] "sm_d36_0"                                      
 [26] "tag_50_1"                                      
 [27] "tag_48_3"                                      
 [28] "tag_50_0"                                      
 [29] "pc_37_4"                                       
 [30] "lpe_20_4"                                      
 [31] "pentadecanoic_acid_c15_0"                      
 [32] "glc_cer_d34_1"                                 
 [33] "ce_18_2"                                       
 [34] "sm_d38_2"                                      
 [35] "lauric_acid_c12_0"                             
 [36] "palmitoleic_acid_16_1"                         
 [37] "pc_o_32_1"                                     
 [38] "dag_34_1"                                      
 [39] "tag_46_2"                                      
 [40] "pc_36_5"                                       
 [41] "pyruvic_acid"                                  
 [42] "ps_41_7"                                       
 [43] "tag_46_1"                                      
 [44] "pe_42_6"                                       
 [45] "arabitol"                                      
 [46] "n_acetyl_galactosamine"                        
 [47] "pc_39_6"                                       
 [48] "pc_37_3"                                       
 [49] "hexose"                                        
 [50] "cis_4_7_10_13_16_19_docosahexaenoic_acid_c22_6"
 [51] "pi_38_4"                                       
 [52] "alpha_ketoglutaric_acid"                       
 [53] "acetic_acid_c2_0"                              
 [54] "dimethyl_succinic_acid"                        
 [55] "linolenic_acid_c18_3"                          
 [56] "cer_d42_1"                                     
 [57] "glc_cer_d42_2"                                 
 [58] "lpc_16_1"                                      
 [59] "allantoin"                                     
 [60] "cer_d40_1"                                     
 [61] "asparagine"                                    
 [62] "pc_38_5"                                       
 [63] "cer_d41_1"                                     
 [64] "erythro_dihydrosphingosine"                    
 [65] "threonine"                                     
 [66] "sm_d33_1"                                      
 [67] "argininosuccinic_acid"                         
 [68] "pc_o_32_0"                                     
 [69] "lpe_22_6"                                      
 [70] "sm_d35_1"                                      
 [71] "pc_o_40_5"                                     
 [72] "aminoadipic_acid"                              
 [73] "glutamic_acid"                     

[1] "adipic_acid"                                   
  [2] "tyrosine"                                      
  [3] "pc_32_1"                                       
  [4] "tag_50_2"                                      
  [5] "pe_40_4"                                       
  [6] "lac_cer_d42_2"                                 
  [7] "tag_48_2"                                      
  [8] "sm_d34_0"                                      
  [9] "aspartic_acid"                                 
 [10] "tag_48_0"                                      
 [11] "pc_32_0"                                       
 [12] "sm_d36_2"                                      
 [13] "tag_50_3"                                      
 [14] "pe_38_2"                                       
 [15] "taurocholic_acid"                              
 [16] "kynurenic_acid"                                
 [17] "deoxyuridine"                                  
 [18] "lpc_20_4"                                      
 [19] "pc_35_2"                                       
 [20] "xanthosine"                                    
 [21] "lpc_18_2"                                      
 [22] "tag_48_1"                                      
 [23] "phenylpyruvic_acid"                            
 [24] "sm_d34_2"                                      
 [25] "sm_d36_0"                                      
 [26] "tag_50_1"                                      
 [27] "tag_48_3"                                      
 [28] "tag_50_0"                                      
 [29] "pc_37_4"                                       
 [30] "lpe_20_4"                                      
 [31] "pentadecanoic_acid_c15_0"                      
 [32] "glc_cer_d34_1"                                 
 [33] "ce_18_2"                                       
 [34] "sm_d38_2"                                      
 [35] "lauric_acid_c12_0"                             
 [36] "palmitoleic_acid_16_1"                         
 [37] "pc_o_32_1"                                     
 [38] "dag_34_1"                                      
 [39] "tag_46_2"                                      
 [40] "pc_36_5"                                       
 [41] "pyruvic_acid"                                  
 [42] "ps_41_7"                                       
 [43] "tag_46_1"                                      
 [44] "pe_42_6"                                       
 [45] "arabitol"                                      
 [46] "n_acetyl_galactosamine"                        
 [47] "pc_39_6"                                       
 [48] "pc_37_3"                                       
 [49] "hexose"                                        
 [50] "cis_4_7_10_13_16_19_docosahexaenoic_acid_c22_6"
 [51] "pi_38_4"                                       
 [52] "alpha_ketoglutaric_acid"                       
 [53] "acetic_acid_c2_0"                              
 [54] "dimethyl_succinic_acid"                        
 [55] "linolenic_acid_c18_3"                          
 [56] "cer_d42_1"                                     
 [57] "glc_cer_d42_2"                                 
 [58] "lpc_16_1"                                      
 [59] "allantoin"                                     
 [60] "cer_d40_1"                                     
 [61] "asparagine"                                    
 [62] "pc_38_5"                                       
 [63] "cer_d41_1"                                     
 [64] "erythro_dihydrosphingosine"                    
 [65] "threonine"                                     
 [66] "sm_d33_1"                                      
 [67] "argininosuccinic_acid"                         
 [68] "pc_o_32_0"                                     
 [69] "lpe_22_6"                                      
 [70] "sm_d35_1"                                      
 [71] "pc_o_40_5"                                     
 [72] "aminoadipic_acid"                              
 [73] "glutamic_acid"                     

In [53]:

roc_control = trainControl(method = 'LOOCV',                
                        number = 1,
                        seed = as.list(rep(1,70)),
                         savePredictions = 'final',        
                        ) # number of folds


roc_pred_stage <- train(group_123 ~ adipic_acid + tyrosine + pc_32_1 + tag_50_2 + pe_40_4,
                 data = metadata_psvd_cir,
                 method = "rf",
                 trControl = roc_control,
                 tuneGrid=data.frame(mtry=1))
roc_pred_stage

Random Forest 

38 samples
 5 predictor
 2 classes: '1', '2' 

No pre-processing
Resampling: Leave-One-Out Cross-Validation 
Summary of sample sizes: 37, 37, 37, 37, 37, 37, ... 
Resampling results:

  Accuracy   Kappa    
  0.8421053  0.6850829

Tuning parameter 'mtry' was held constant at a value of 1